In [7]:
import os, mlflow
from dotenv import load_dotenv

load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/priscila.cervantes@iteso.mx/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

2025/10/21 20:45:49 INFO mlflow.tracking.fluent: Experiment with name '/Users/priscila.cervantes@iteso.mx/nyc-taxi-experiments' does not exist. Creating a new experiment.


## Train Model

In [8]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [9]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [13]:
df_train = read_dataframe('C:/Users/prisc/apps/nyc-taxi-predictions-2025/data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('C:/Users/prisc/apps/nyc-taxi-predictions-2025/data/green_tripdata_2025-02.parquet')

In [14]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [15]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

In [16]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

## Optimización de hiperparámetros usando Optuna

In [18]:
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

## Función objetivo

In [20]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna
#    - Recibe un `trial`, que se usa para proponer hiperparámetros.
#    - Entrena un modelo con esos hiperparámetros.
#    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
#    - Abrimos un run anidado de MLflow para registrar cada trial.
# ------------------------------------------------------------
def objective(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",  
        "seed": 42,                      
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "xgboost")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento con early stopping en el conjunto de validación
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10,
        )

        # Predicción y métrica en validación
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # Optuna minimiza el valor retornado
    return rmse

## Flujo de búsqueda

In [22]:
mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=10)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=10,
    )

    # Evaluar y registrar la métrica final en validación
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.xgboost.log_model(
        booster,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-21 20:59:47,011] A new study created in memory with name: no-name-d0325820-9a0e-4eb7-9d23-2639e0b041f8


[0]	validation-rmse:5.98960
[1]	validation-rmse:5.92413
[2]	validation-rmse:5.92287
[3]	validation-rmse:5.92324
[4]	validation-rmse:5.92483
[5]	validation-rmse:5.92489
[6]	validation-rmse:5.92498
[7]	validation-rmse:5.92343
[8]	validation-rmse:5.92347
[9]	validation-rmse:5.92467
[10]	validation-rmse:5.92456
[11]	validation-rmse:5.92460
[12]	validation-rmse:5.92458


c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [20:59:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:00:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:00:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-21 21:00:05,676] Trial 0 finished with value: 5.924581902007077 and parameters: {'max_depth': 40, 'learning_rate': 0.8625543817410922, 'reg_alpha': 0.12593061066249622, 'reg_lambda': 0.049454235173237264, 'min_child_weight': 0.6866535292359801}. Best is trial 0 with value: 5.924581902007077.


🏃 View run omniscient-donkey-53 at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/2168270428826736/runs/1810bfd389eb4cda8404f271e7bc536a
🧪 View experiment at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/2168270428826736
[0]	validation-rmse:8.79510
[1]	validation-rmse:8.50586
[2]	validation-rmse:8.24215
[3]	validation-rmse:8.00493
[4]	validation-rmse:7.78830
[5]	validation-rmse:7.59575
[6]	validation-rmse:7.41983
[7]	validation-rmse:7.26129
[8]	validation-rmse:7.11270
[9]	validation-rmse:6.98583
[10]	validation-rmse:6.87056
[11]	validation-rmse:6.76533
[12]	validation-rmse:6.67029
[13]	validation-rmse:6.58378
[14]	validation-rmse:6.50773
[15]	validation-rmse:6.43768
[16]	validation-rmse:6.37633
[17]	validation-rmse:6.32116
[18]	validation-rmse:6.27142
[19]	validation-rmse:6.22765
[20]	validation-rmse:6.18963
[21]	validation-rmse:6.15802
[22]	validation-rmse:6.12794
[23]	validation-rmse:6.10295
[24]	validation-rmse:6.07940
[25]	validation-rmse:6.05863
[2

...

c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:04:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-21 21:05:16,436] Trial 9 finished with value: 5.999083279452748 and parameters: {'max_depth': 68, 'learning_rate': 0.1268351874747755, 'reg_alpha': 0.05394836382863035, 'reg_lambda': 0.03814164293595655, 'min_child_weight': 0.7706028272535065}. Best is trial 8 with value: 5.862230443089824.


🏃 View run chill-crane-139 at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/2168270428826736/runs/99e0ad87806f4714bb76944a00b56e9b
🧪 View experiment at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/2168270428826736
[0]	validation-rmse:7.96125
[1]	validation-rmse:7.19956
[2]	validation-rmse:6.69631
[3]	validation-rmse:6.37886
[4]	validation-rmse:6.17661
[5]	validation-rmse:6.05039
[6]	validation-rmse:5.98463
[7]	validation-rmse:5.93473
[8]	validation-rmse:5.90786
[9]	validation-rmse:5.89293
[10]	validation-rmse:5.88940
[11]	validation-rmse:5.88270
[12]	validation-rmse:5.87865
[13]	validation-rmse:5.87724
[14]	validation-rmse:5.87452
[15]	validation-rmse:5.87431
[16]	validation-rmse:5.87284
[17]	validation-rmse:5.87217
[18]	validation-rmse:5.87229
[19]	validation-rmse:5.87087
[20]	validation-rmse:5.87091
[21]	validation-rmse:5.87085
[22]	validation-rmse:5.87091
[23]	validation-rmse:5.86997
[24]	validation-rmse:5.86922
[25]	validation-rmse:5.86934
[26]	va

c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:169: UserWarning: [21:05:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:05:28 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:05:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


🏃 View run XGBoost Hyperparameter Optimization (Optuna) at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/2168270428826736/runs/1caa88fd4fa8439c9a5fca8f428589a5
🧪 View experiment at: https://dbc-35a009eb-7ad0.cloud.databricks.com/ml/experiments/2168270428826736


## Registrar modelo en 'Model Registry'

### Método 1: Manual

In [23]:
model_name = "workspace.default.nyc-taxi-model"

In [24]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="workspace.default.nyc-taxi-model"
)

Successfully registered model 'workspace.default.nyc-taxi-model'.
2025/10/21 21:14:50 WARNING mlflow.tracking._model_registry.fluent: Run with id 1caa88fd4fa8439c9a5fca8f428589a5 has no artifacts at artifact path 'model', registering model based on models:/m-b79602fe0e034844ab0f8dbe9d70de15 instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '1' of model 'workspace.default.nyc-taxi-model'.


### Método 2: Automático

In [25]:
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    order_by=["metrics.rmse ASC"],
    output_format="list"
)

# Obtener el mejor run
if len(runs) > 0:
    best_run = runs[0]
    print("🏆 Champion Run encontrado:")
    print(f"Run ID: {best_run.info.run_id}")
    print(f"RMSE: {best_run.data.metrics['rmse']}")
    print(f"Params: {best_run.data.params}")
else:
    print("⚠️ No se encontraron runs con métrica RMSE.")

🏆 Champion Run encontrado:
Run ID: 94233b2dec3445f18950c0e1de6f1936
RMSE: 5.862230443089824
Params: {'custom_metric': 'None', 'early_stopping_rounds': '10', 'learning_rate': '0.21992487468175848', 'max_depth': '15', 'maximize': 'None', 'min_child_weight': '1.0357439143907545', 'num_boost_round': '100', 'objective': 'reg:squarederror', 'reg_alpha': '0.007731550026907306', 'reg_lambda': '0.23377457337376373', 'seed': '42', 'verbose_eval': 'True'}


In [28]:
run_id = best_run.info.run_id

In [29]:
result = mlflow.register_model(
    model_uri=f"runs:/{best_run.info.run_id}/model",
    name=model_name
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/21 21:23:56 WARNING mlflow.tracking._model_registry.fluent: Run with id 94233b2dec3445f18950c0e1de6f1936 has no artifacts at artifact path 'model', registering model based on models:/m-d31189e8f1c1476ea3647dce3e689985 instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '2' of model 'workspace.default.nyc-taxi-model'.


## Asignar alias

In [30]:
from mlflow import MlflowClient

client = MlflowClient()

In [31]:
model_version = result.version
new_alias = "Champion"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

In [32]:
from datetime import datetime

date = datetime.today()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1761103441005, current_stage=None, deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 2 was transitioned to Champion on 2025-10-21 21:25:48.197178', last_updated_timestamp=1761103548031, metrics=[<Metric: dataset_digest='', dataset_name='', key='best_iteration', model_id='m-d31189e8f1c1476ea3647dce3e689985', run_id='94233b2dec3445f18950c0e1de6f1936', step=0, timestamp=1761102248277, value=80.0>,
 <Metric: dataset_digest='', dataset_name='', key='rmse', model_id='m-d31189e8f1c1476ea3647dce3e689985', run_id='94233b2dec3445f18950c0e1de6f1936', step=0, timestamp=1761102251839, value=5.862230443089824>,
 <Metric: dataset_digest='', dataset_name='', key='stopped_iteration', model_id='m-d31189e8f1c1476ea3647dce3e689985', run_id='94233b2dec3445f18950c0e1de6f1936', s

## Obteniendo modelos de model registry

In [33]:
import mlflow.pyfunc

model_version_uri = f"models:/{model_name}@Champion"

champion_version = mlflow.pyfunc.load_model(model_version_uri)
champion_version.predict(X_val)

c:\Users\prisc\apps\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [21:28:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


array([ 5.0574026, 24.71894  , 28.723047 , ..., 20.382122 , 13.796955 ,
       22.728203 ], shape=(44218,), dtype=float32)